<a href="https://colab.research.google.com/github/Dhanraj7573/Python-Projects/blob/main/Stress_Testing_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Stress Test Scenarios Tool (ONE-SHOT NOTEBOOK / SCRIPT)
# - Creates an example portfolio CSV
# - Lets you upload your own portfolio CSV (in Google Colab) OR use a local path
# - Prices holdings (if using shares) with yfinance
# - Runs multiple stress scenarios and prints a clean report
#
# Portfolio CSV formats supported:
#   A) Shares-based (recommended):
#        ticker,asset_class,shares
#        SPY,equity,10
#        AGG,bond,30
#
#   B) Value-based:
#        ticker,asset_class,value
#        SPY,equity,7000
#        AGG,bond,3000

import os
import sys
import subprocess
from typing import Dict, Tuple

# ----------------------------
# 0) Ensure dependencies
# ----------------------------
def ensure_packages():
    needed = ["pandas", "numpy", "yfinance"]
    missing = []
    for pkg in needed:
        try:
            __import__(pkg)
        except ImportError:
            missing.append(pkg)

    if missing:
        print("Installing missing packages:", missing)
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + missing)

ensure_packages()

import pandas as pd
import numpy as np
import yfinance as yf


# ----------------------------
# 1) Create an example portfolio CSV
# ----------------------------
EXAMPLE_CSV = "portfolio_example.csv"

example_portfolio = pd.DataFrame(
    [
        {"ticker": "SPY", "asset_class": "equity", "shares": 8},   # US equities
        {"ticker": "QQQ", "asset_class": "equity", "shares": 5},   # US tech-heavy
        {"ticker": "AGG", "asset_class": "bond",   "shares": 40},  # US bonds
        {"ticker": "GLD", "asset_class": "gold",   "shares": 6},   # gold
        {"ticker": "BIL", "asset_class": "cash",   "shares": 60},  # cash-like T-bills ETF
    ]
)
example_portfolio.to_csv(EXAMPLE_CSV, index=False)
print(f"✅ Created example portfolio file: {EXAMPLE_CSV}")
print(example_portfolio)


# ----------------------------
# 2) Load portfolio (upload in Colab or use a local file)
# ----------------------------
def is_colab() -> bool:
    try:
        import google.colab  # noqa: F401
        return True
    except Exception:
        return False

def choose_portfolio_file(default_path: str = EXAMPLE_CSV) -> str:
    """
    In Colab: prompts upload and uses uploaded CSV if provided.
    Locally: uses default_path (or you can set PORTFOLIO_CSV env var).
    """
    env_path = os.getenv("PORTFOLIO_CSV")
    if env_path and os.path.exists(env_path):
        print(f"Using PORTFOLIO_CSV from env: {env_path}")
        return env_path

    if is_colab():
        print("\n📤 Colab detected. Upload your portfolio CSV now (or cancel to use the example).")
        try:
            from google.colab import files
            uploaded = files.upload()
            if uploaded:
                # Use the first uploaded file
                fname = next(iter(uploaded.keys()))
                print(f"Using uploaded file: {fname}")
                return fname
        except Exception as e:
            print("Upload step skipped/failed, using example CSV. Reason:", e)

    # Fallback
    print(f"Using default portfolio file: {default_path}")
    return default_path

PORTFOLIO_CSV = choose_portfolio_file()


# ----------------------------
# 3) Portfolio parsing + pricing
# ----------------------------
def load_portfolio(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    required = {"ticker", "asset_class"}
    if not required.issubset(set(df.columns)):
        raise ValueError(f"CSV must include columns: {sorted(required)}. Found: {list(df.columns)}")

    df["ticker"] = df["ticker"].astype(str).str.upper().str.strip()
    df["asset_class"] = df["asset_class"].astype(str).str.lower().str.strip()

    has_shares = "shares" in df.columns
    has_value = "value" in df.columns

    if not has_shares and not has_value:
        raise ValueError("CSV must include either 'shares' or 'value' column.")

    if has_shares:
        df["shares"] = pd.to_numeric(df["shares"], errors="coerce")
        df = df.dropna(subset=["shares"])
    if has_value:
        df["value"] = pd.to_numeric(df["value"], errors="coerce")
        df = df.dropna(subset=["value"]) if not has_shares else df  # allow both

    df = df.reset_index(drop=True)
    return df

def add_prices_and_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    If 'value' already exists and shares not provided, keep as is.
    If 'shares' exists, fetch latest prices and compute value.
    """
    out = df.copy()

    if "shares" not in out.columns:
        # value-based portfolio
        return out

    tickers = out["ticker"].unique().tolist()
    px = yf.download(tickers, period="10d", auto_adjust=True, progress=False)["Close"]

    # Handle single ticker
    last_prices = {}
    if isinstance(px, pd.Series):
        series = px.dropna()
        if not series.empty:
            last_prices[tickers[0]] = float(series.iloc[-1])
    else:
        for t in tickers:
            if t in px.columns:
                s = px[t].dropna()
                if not s.empty:
                    last_prices[t] = float(s.iloc[-1])

    out["price"] = out["ticker"].map(last_prices)
    missing = out[out["price"].isna()]["ticker"].unique().tolist()
    if missing:
        print("\n⚠️ Could not fetch prices for:", missing)
        print("   These rows will be dropped (check ticker symbols or try again).")

    out = out.dropna(subset=["price"]).copy()
    out["value"] = out["shares"] * out["price"]
    return out


# ----------------------------
# 4) Stress scenario engine
# ----------------------------
def apply_scenario(portfolio: pd.DataFrame, shocks: Dict[str, float], name: str) -> Tuple[pd.DataFrame, Dict]:
    """
    shocks can contain:
      - asset class shocks: {"equity": -0.2, "bond": -0.05}
      - ticker overrides: {"SPY": -0.25}  (if a ticker key is present, it overrides asset_class)
    """
    df = portfolio.copy()

    def get_shock(row):
        t = row["ticker"]
        ac = row["asset_class"]
        if t in shocks:
            return float(shocks[t])
        return float(shocks.get(ac, 0.0))

    df["shock"] = df.apply(get_shock, axis=1)
    df["stressed_value"] = df["value"] * (1.0 + df["shock"])
    df["pnl"] = df["stressed_value"] - df["value"]
    df["pnl_pct"] = np.where(df["value"] != 0, df["pnl"] / df["value"], np.nan)
    df["scenario"] = name

    total_value = float(df["value"].sum())
    stressed_total = float(df["stressed_value"].sum())
    total_pnl = float(df["pnl"].sum())
    total_pnl_pct = total_pnl / total_value if total_value else np.nan

    summary = {
        "scenario": name,
        "portfolio_value": total_value,
        "stressed_value": stressed_total,
        "pnl": total_pnl,
        "pnl_pct": total_pnl_pct,
    }
    return df, summary

def run_stress_tests(portfolio: pd.DataFrame, scenarios: Dict[str, Dict[str, float]], top_n: int = 5):
    all_details = []
    summaries = []

    for name, shocks in scenarios.items():
        det, summ = apply_scenario(portfolio, shocks, name=name)
        all_details.append(det)
        summaries.append(summ)

    details_df = pd.concat(all_details, ignore_index=True)
    summary_df = pd.DataFrame(summaries).sort_values("pnl").reset_index(drop=True)

    # Pretty print summary
    def fmt_money(x): return f"{x:,.2f}"
    def fmt_pct(x): return f"{x:.2%}" if pd.notna(x) else "NA"

    print("\n==============================")
    print("✅ STRESS TEST SUMMARY")
    print("==============================")
    print(summary_df.to_string(
        index=False,
        formatters={
            "portfolio_value": fmt_money,
            "stressed_value": fmt_money,
            "pnl": fmt_money,
            "pnl_pct": fmt_pct,
        }
    ))

    # Drivers for worst scenario
    worst_name = summary_df.iloc[0]["scenario"]
    worst = details_df[details_df["scenario"] == worst_name].copy().sort_values("pnl")  # most negative first

    print("\n==============================")
    print(f"🔻 TOP LOSS DRIVERS (Worst Scenario: {worst_name})")
    print("==============================")
    cols = ["ticker", "asset_class", "value", "shock", "pnl"]
    print(worst.head(top_n)[cols].to_string(
        index=False,
        formatters={
            "value": fmt_money,
            "shock": lambda x: f"{x:.1%}",
            "pnl": fmt_money,
        }
    ))

    return summary_df, details_df


# ----------------------------
# 5) Define scenarios (edit freely)
# ----------------------------
scenarios = {
    "Equities -20%, Bonds -5%": {
        "equity": -0.20,
        "bond": -0.05,
        "gold": 0.05,
        "cash": 0.00,
    },
    "Rates spike (Bonds -12%, Equities -8%)": {
        "equity": -0.08,
        "bond": -0.12,
        "gold": -0.02,
        "cash": 0.01,
    },
    "Risk-off (Equities -30%, Bonds +3%)": {
        "equity": -0.30,
        "bond": 0.03,
        "gold": 0.10,
        "cash": 0.00,
    },
    "Mild selloff (Equities -10%, Bonds -2%)": {
        "equity": -0.10,
        "bond": -0.02,
        "gold": 0.01,
        "cash": 0.00,
    },
    # Example of ticker override: SPY gets hit harder than other equities
    "Equities -20% but SPY -25% (ticker override)": {
        "equity": -0.20,
        "bond": -0.05,
        "gold": 0.05,
        "cash": 0.00,
        "SPY": -0.25,
    },
}


# ----------------------------
# 6) Run end-to-end
# ----------------------------
portfolio_raw = load_portfolio(PORTFOLIO_CSV)
portfolio = add_prices_and_values(portfolio_raw)

if portfolio.empty:
    raise SystemExit("Portfolio is empty after loading/pricing. Check your CSV and tickers.")

portfolio["weight"] = portfolio["value"] / portfolio["value"].sum()

print("\n==============================")
print("📌 PORTFOLIO (priced + valued)")
print("==============================")
print(portfolio[["ticker", "asset_class", "shares" if "shares" in portfolio.columns else "value", "price" if "price" in portfolio.columns else "value", "value", "weight"]]
      .to_string(index=False, formatters={"value": "{:,.2f}".format, "weight": lambda x: f"{x:.2%}"}))

summary_df, details_df = run_stress_tests(portfolio, scenarios, top_n=7)

# Save outputs
summary_df.to_csv("stress_test_summary.csv", index=False)
details_df.to_csv("stress_test_details.csv", index=False)
print("\n💾 Saved files: stress_test_summary.csv, stress_test_details.csv")
print("Tip: edit scenarios dict or portfolio_example.csv to try different setups.")


✅ Created example portfolio file: portfolio_example.csv
  ticker asset_class  shares
0    SPY      equity       8
1    QQQ      equity       5
2    AGG        bond      40
3    GLD        gold       6
4    BIL        cash      60

📤 Colab detected. Upload your portfolio CSV now (or cancel to use the example).


Using default portfolio file: portfolio_example.csv

📌 PORTFOLIO (priced + valued)
ticker asset_class  shares      price    value weight
   SPY      equity       8 682.849976 5,462.80 26.40%
   QQQ      equity       5 601.299988 3,006.50 14.53%
   AGG        bond      40 101.000000 4,040.00 19.53%
   GLD        gold       6 448.200012 2,689.20 13.00%
   BIL        cash      60  91.519997 5,491.20 26.54%

✅ STRESS TEST SUMMARY
                                    scenario portfolio_value stressed_value       pnl pnl_pct
         Risk-off (Equities -30%, Bonds +3%)       20,689.70      18,539.03 -2,150.67 -10.39%
Equities -20% but SPY -25% (ticker override)       20,689.70      18,655.16 -2,034.54  -9.83%
                    Equities -20%, Bonds -5%       20,689.70      18,928.30 -1,761.40  -8.51%
      Rates spike (Bonds -12%, Equities -8%)       20,689.70      19,528.48 -1,161.22  -5.61%
     Mild selloff (Equities -10%, Bonds -2%)       20,689.70      19,788.86   -900.84  -4.35%

🔻 TOP